In [27]:
import yfinance as yf
import pandas as pd
from datetime import timedelta
from fredapi import Fred


def build_event_panel(fomc_dates, tickers, vol_window=5):
    rows = []

    start = fomc_dates.min() - timedelta(days=365)
    end = fomc_dates.max() + timedelta(days=10)

    # Market benchmark
    market_close = yf.download(
        "^GSPC", start=start, end=end, auto_adjust=True, progress=False
    )["Close"]

     # Must be series
    if isinstance(market_close, pd.DataFrame):
        market_close = market_close.iloc[:, 0]

    market_ret = market_close.pct_change()

    for ticker in tickers:
        print(f"Processing {ticker}...")

        asset_close = yf.download(
            ticker, start=start, end=end, auto_adjust=True, progress=False
        )["Close"]

        # Must be series
        if isinstance(asset_close, pd.DataFrame):
            asset_close = asset_close.iloc[:, 0]

        asset_ret = asset_close.pct_change()

        for event_date in fomc_dates:
            if event_date > asset_ret.index.max():
                continue

            loc = asset_ret.index.searchsorted(event_date)
            if loc >= len(asset_ret) or loc < vol_window:
                continue

            actual_date = asset_ret.index[loc]

            # --- scalar market return ---
            market_val = market_ret.loc[actual_date]
            if isinstance(market_val, pd.Series):
                market_val = market_val.iloc[0]

            asset_val = asset_ret.iloc[loc]
            if isinstance(asset_val, pd.Series):
                asset_val = asset_val.iloc[0]

            ar = asset_val - market_val

            # --- scalar volatility ---
            vol_window_ret = asset_ret.iloc[loc - vol_window:loc]
            vol = vol_window_ret.std()
            if isinstance(vol, pd.Series):
                vol = vol.iloc[0]

            rows.append({
                "event_date": actual_date,
                "ticker": ticker,
                "abnormal_return": float(ar),
                "volatility_5d": float(vol),
                "label": int(ar > 0)
            })

    return pd.DataFrame(rows)


In [28]:
# FOMC dates
fomc_dates = pd.to_datetime([
    '2015-12-16', '2016-12-14', '2017-03-15', '2017-06-14',
    '2017-09-20', '2017-12-13', '2018-03-21', '2018-06-13',
    '2018-09-26', '2018-12-19', '2019-07-31', '2019-09-18',
    '2020-03-15', '2020-04-29', '2020-06-10', '2020-09-16',
    '2021-03-17', '2021-06-16', '2022-03-16', '2022-05-04',
    '2022-06-15', '2022-07-27', '2022-09-21', '2022-11-02',
    '2022-12-14', '2023-02-01', '2023-03-22', '2023-05-03',
    '2023-06-14', '2023-07-26', '2023-09-20', '2023-11-01',
    '2023-12-13', '2024-01-31', '2024-03-20', '2024-05-01',
    '2024-06-12'
])

tickers = ["SPY", "QQQ", "IWM", "XLF"]

df = build_event_panel(fomc_dates, tickers)

print(df.head())
print(df.isna().sum())
print(df.groupby("ticker").size())


Processing SPY...
Processing QQQ...
Processing IWM...
Processing XLF...
  event_date ticker  abnormal_return  volatility_5d  label
0 2015-12-16    SPY         0.000117       0.011856      1
1 2016-12-14    SPY        -0.000137       0.005306      0
2 2017-03-15    SPY         0.000279       0.002837      1
3 2017-06-14    SPY        -0.000272       0.002442      0
4 2017-09-20    SPY        -0.000275       0.000919      0
event_date         0
ticker             0
abnormal_return    0
volatility_5d      0
label              0
dtype: int64
ticker
IWM    37
QQQ    37
SPY    37
XLF    37
dtype: int64


In [30]:
fomc_dates = pd.to_datetime([
'2015-12-16', '2016-12-14', '2017-03-15', '2017-06-14',
'2017-09-20', '2017-12-13', '2018-03-21', '2018-06-13',
'2018-09-26', '2018-12-19', '2019-07-31', '2019-09-18',
'2020-03-15', '2020-04-29', '2020-06-10', '2020-09-16',
'2021-03-17', '2021-06-16', '2022-03-16', '2022-05-04',
'2022-06-15', '2022-07-27', '2022-09-21', '2022-11-02',
'2022-12-14', '2023-02-01', '2023-03-22', '2023-05-03',
'2023-06-14', '2023-07-26', '2023-09-20', '2023-11-01',
'2023-12-13', '2024-01-31', '2024-03-20', '2024-05-01',
'2024-06-12'
])

tickers = ["SPY", "QQQ", "IWM", "XLF"]

df = build_event_panel(fomc_dates, tickers)

print(df.head())
print(df.isna().sum())
print(df.groupby("ticker").size())
df.head()

Processing SPY...
Processing QQQ...
Processing IWM...
Processing XLF...
  event_date ticker  abnormal_return  volatility_5d  label
0 2015-12-16    SPY         0.000117       0.011856      1
1 2016-12-14    SPY        -0.000137       0.005306      0
2 2017-03-15    SPY         0.000279       0.002837      1
3 2017-06-14    SPY        -0.000272       0.002442      0
4 2017-09-20    SPY        -0.000275       0.000919      0
event_date         0
ticker             0
abnormal_return    0
volatility_5d      0
label              0
dtype: int64
ticker
IWM    37
QQQ    37
SPY    37
XLF    37
dtype: int64


,event_date,ticker,abnormal_return,volatility_5d,label
0,2015-12-16,SPY,0.000117,0.011856,1
1,2016-12-14,SPY,-0.000137,0.005306,0
2,2017-03-15,SPY,0.000279,0.002837,1
3,2017-06-14,SPY,-0.000272,0.002442,0
4,2017-09-20,SPY,-0.000275,0.000919,0
